In [26]:
import os
import msal
from msal.application import ConfidentialClientApplication
from dotenv import load_dotenv
import jwt
import json
import requests
from datetime import datetime, timedelta
from typing import Optional, Dict, List

# Define the target directory
target_directory = r'C:\Users\pablosal\Desktop\sharepoint-indexing-azure-cognitive-search'

# Load .env file
load_dotenv()

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

#load logging 
from utils.ml_logging import get_logger
logger = get_logger()

Directory changed to C:\Users\pablosal\Desktop\sharepoint-indexing-azure-cognitive-search


In [27]:
import asyncio
from azure.identity.aio import ClientSecretCredential
from msgraph import GraphServiceClient

In [28]:
import asyncio
from azure.identity.aio import ClientSecretCredential
from msgraph import GraphServiceClient

# Get the variables
tenantID = os.getenv('TENANT_ID')
clientID = os.getenv('CLIENT_ID')
clientSecret = os.getenv('CLIENT_SECRET')
graphURI = 'https://graph.microsoft.com'
authority = 'https://login.microsoftonline.com/' + tenantID
scope = ['https://graph.microsoft.com/.default']

credential = ClientSecretCredential(
    tenantID,
    clientID,
    clientSecret
)
scopes = ['https://graph.microsoft.com/.default']
client = GraphServiceClient(credentials=credential, scopes=scopes)

In [29]:
async def get_applications():
    apps = await client.applications.get()
    if apps and apps.value:
        for app in apps.value:
            print(app.id)

In [30]:
async def get_users():
    users = await client.users.get()
    if users and users.value:
        for user in users.value:
            print(user.id, user.display_name, user.mail)

In [31]:
from msgraph.generated.users.item.messages.messages_request_builder import MessagesRequestBuilder

async def get_user_messages():
    request_config = MessagesRequestBuilder.MessagesRequestBuilderGetRequestConfiguration(
    )
    request_config.headers.add("prefer", "outlook.body-content-type=text")

    messages = await (client.users.by_user_id('USER_ID')
                    .messages
                    .get(request_configuration=request_config))
    if messages and messages.value:
        for msg in messages.value:
            print(msg.subject, msg.id, msg.from_)

In [32]:
async def get_drives():
    drives = await client.drives.get()
    if drives and drives.value:
        for drive in drives.value:
            print(drive.id, drive.drive_type, drive.name, drive.description, drive.web_url)

In [33]:
await get_applications()

2d3e4338-0a69-4904-8f80-cc8878ffa687
6fc47a30-0f2b-4ac1-b792-4326f36df354
bfaf9e16-6ea2-4795-ac79-b2338b20078a


In [34]:
await get_drives()

b!LD-80aZbckOQCqgs72olAj3fTteFBCRNugY9EbqD_eEv4NV4KCpRTYwd3M6492Tg documentLibrary Documents  https://mngenvmcap747548.sharepoint.com/Shared%20Documents


In [14]:
await get_users()

ea78aab4-722b-48f7-8a37-7b81b4ba0103 System Administrator admin@MngEnvMCAP747548.onmicrosoft.com
f08207bf-08e2-42cf-ba4c-ac9a99c2103d Microsoft Service Account None
4e6b32a4-9233-4256-8e97-94e5584c5886 Pablo Salvador Lopez pablosal@microsoft.com
1e8fea8e-64f2-427d-a364-8bb18eb9ccc3 pablosalvadorlopez11 pablosalvadorlopez11@gmail.com


In [35]:
async def get_sites():
    sites = await client.sites.get()
    if sites and sites.value:
        for site in sites.value:
            print(site)

In [36]:
await get_sites()

Site(additional_data={}, id='mngenvmcap747548.sharepoint.com,877fe60f-a62d-4ed1-8eda-af543c437d2c,ac47d8a7-cd54-4344-bd9d-26ada5a075c0', odata_type='#microsoft.graph.site', created_by=None, created_by_user=None, created_date_time=DateTime(2023, 10, 18, 14, 20, 27, tzinfo=Timezone('UTC')), description=None, e_tag=None, last_modified_by=None, last_modified_by_user=None, last_modified_date_time=None, name='Contoso', parent_reference=None, web_url='https://mngenvmcap747548.sharepoint.com/sites/Contoso', analytics=None, columns=None, content_types=None, display_name='Contoso', drive=None, drives=None, error=None, external_columns=None, is_personal_site=False, items=None, lists=None, onenote=None, operations=None, permissions=None, root=Root(additional_data={}, odata_type=None), sharepoint_ids=None, site_collection=SiteCollection(additional_data={}, data_location_code=None, hostname='mngenvmcap747548.sharepoint.com', odata_type=None, root=None), sites=None, term_store=None, term_stores=None)

In [40]:
site_domain = "mngenvmcap747548.sharepoint.com"
site_name= "Contoso"

In [47]:
endpoint = 'https://graph.microsoft.com/v1.0/sites/' + f"{site_domain}" + ':/sites/' +  f"{site_name}" + ':/'

In [56]:
sites = client.sites.with_url(endpoint)

In [63]:
sites = await client.sites.get()

In [43]:
async def get_sites():
    sites = client.sites.with_url(endpoint)
    if sites and sites.value:
        for site in sites.value:
            print(site.id)